In [ ]:
import prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
raw_data = pd.read_csv('../data/demo_data.csv')
service_list = raw_data.columns.to_list()[1:]
raw_data = raw_data.rename(columns={'date': 'ds'})
# raw_data['ds'] = pd.to_datetime(raw_data['ds'])
raw_data

In [ ]:
train_data = raw_data.iloc[-28*288:, :]
print('train size: ', train_data.values.shape)

In [ ]:
start_time = '2023-08-18 00:00:00'
end_time = '2023-08-31 23:55:00'
time_range = pd.date_range(start=start_time, end=end_time, freq='5min')

# 创建DataFrame
future = pd.DataFrame({'ds': time_range})
future

In [ ]:
future.iloc[:7*288]

In [ ]:
pred_list = []
for svc in service_list:
    print(svc)
    svc_pred_list = []
    for i in range(14):
        # print(i)
        # print(train_data.iloc[i*288:(i+14)*288, :].loc[:, ['ds', svc]].values.shape)
        # step1 创建模型并训练
        model = prophet.Prophet(changepoint_prior_scale=0.01)
        model.fit(train_data.iloc[i*288:(i+14)*288, :].loc[:, ['ds', svc]].rename(columns={svc: 'y'}))
        # step2 预测
        fcst = model.predict(future.iloc[i*288:(i+1)*288])['yhat'].values
        svc_pred_list.append(fcst)
    svc_pred = np.concatenate(svc_pred_list)
    pred_list.append(svc_pred)
    # # step3 计算mape
    # merged_df = pd.merge(test_data.loc[test_data['group_lable'] == group_lable, ['ds', 'y']], fcst[['ds', 'yhat']], on='ds', how='inner')
    # merged_df['mape'] = (abs(merged_df['y'] - merged_df['yhat']) / merged_df['y']) * 100
    # mape = merged_df['mape'].mean()
    # # 显示MAPE
    # mape_list.append(mape)
    # print(group_lable, 'MAPE:', mape, '%')
    # # step4 画图保存
    # plt.figure(figsize=(18, 6))
    # plt.title(group_lable)
    # plt.plot(test_data.loc[test_data['group_lable'] == group_lable, 'ds'] ,test_data.loc[test_data['group_lable'] == group_lable,'y'], label='metrics')
    # plt.plot(fcst['ds'], fcst['yhat'], label='prophet predict')
    # plt.legend()
    # plt.savefig(save_dir+group_lable+'_prophet.svg', format='svg')

In [ ]:
print(len(pred_list))

In [ ]:
pred = np.vstack(pred_list)

In [ ]:
print(pred.shape)